In [16]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from scipy.integrate import solve_ivp

In [17]:
animation.writer = animation.writers['ffmpeg']

In [38]:
# Parameters
g = 9.81  # acceleration due to gravity (m/s^2)
L1 = 1.0  # length of the first rod (m)
L2 = 1.0  # length of the second rod (m)
m1 = 1.0  # mass of the first bob (kg)
m2 = 1.0  # mass of the second bob (kg)
t_final = 20
exp_name = 'start_half_vertical'

# Initial conditions
theta1_init = np.pi
theta2_init = 0
z1_init = 0.0            # initial angular velocity of the first pendulum (rad/s)
z2_init = 0.0            # initial angular velocity of the second pendulum (rad/s)

# Equations of motion
def derivatives(t, state):
    theta1, z1, theta2, z2 = state
    delta = theta2 - theta1

    denominator1 = (m1 + m2) * L1 - m2 * L1 * np.cos(delta) ** 2
    denominator2 = (L2 / L1) * denominator1

    dtheta1_dt = z1
    dz1_dt = (
        (m2 * L1 * z1 ** 2 * np.sin(delta) * np.cos(delta)
         + m2 * g * np.sin(theta2) * np.cos(delta)
         + m2 * L2 * z2 ** 2 * np.sin(delta)
         - (m1 + m2) * g * np.sin(theta1))
        / denominator1
    )
    dtheta2_dt = z2
    dz2_dt = (
        (-m2 * L2 * z2 ** 2 * np.sin(delta) * np.cos(delta)
         + (m1 + m2) * g * np.sin(theta1) * np.cos(delta)
         - (m1 + m2) * L1 * z1 ** 2 * np.sin(delta)
         - (m1 + m2) * g * np.sin(theta2))
        / denominator2
    )

    return np.array([dtheta1_dt, dz1_dt, dtheta2_dt, dz2_dt])


state_0 = np.array([theta1_init, z1_init, theta2_init, z2_init])
t = np.linspace(0, t_final, 2000)  # time array

# Solve the system
solution = solve_ivp(derivatives, (0, t_final), state_0, t_eval=t)
theta1, theta2 = solution.y[0], solution.y[2]

# Convert to Cartesian coordinates
x1 = L1 * np.sin(theta1)
y1 = -L1 * np.cos(theta1)
x2 = x1 + L2 * np.sin(theta2)
y2 = y1 - L2 * np.cos(theta2)

# Animation
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(-(L1+L2)*1.1, (L1+L2)*1.1)
ax.set_ylim(-(L1+L2)*1.1, (L1+L2)*1.1)
ax.set_aspect('equal')
line, = ax.plot([], [], 'o-', lw=2, color='blue')

def update(frame):
    line.set_data([0, x1[frame], x2[frame]], [0, y1[frame], y2[frame]])
    return line,

ani = FuncAnimation(fig, update, frames=len(t), interval=(t_final / 2), blit=True)
ani.save(f'../animations/{exp_name}.mp4')
plt.close(fig)

<video src='../animations/start_half_vertical.mp4' controls>